In [2]:
# import dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from datetime import datetime

In [3]:
# Import the requests library
import requests

#import the API key
from config import weather_api_key

In [4]:
# starting URL for Wather Map API Call
url="http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID="+weather_api_key

In [5]:
# create a set of random latitudes and longitudes combinations
lats=np.random.uniform(low=-90.000, high=90.000, size=2000)
lngs=np.random.uniform(low=-180.000, high=180.000, size=2000)
lats_lngs=zip(lats,lngs)
lats_lngs

In [6]:
# Import Citipy
from citipy import citipy

#Add lats and long to list
coordinates=list(lats_lngs)

In [7]:
# Create a list for holding the cities
cities=[]
#identify the nearest city for each lat and long combination
for coordinate in coordinates:
    city=citipy.nearest_city(coordinate[0], coordinate[1]).city_name
    
    #If the city is unique, then we will add it to the cities list
    if city not in cities:
        cities.append(city)
#Print the city count to confirm sufficient count
len(cities)

725

In [18]:
#Create an empty list to hold the weather data
city_data=[]
#print the beginning of the logging
print("Beginning Data Retrieval       ")
print("-------------------------------")

#Create counters
record_count=1
set_count=1
#Loop through all the cities in our list
for i, city in enumerate(cities):
    
    #Group cities in sets of 50 for logging purposes
    if (i%50==0 and i>=50):
        set_count+=1
        record_count=1
        time.sleep(60)
        
    #Create endpoint URL with each city
    city_url=url+"&q="+ city.replace(" ","+")
    
    #Log the URL, record and set numbers and the city
    print(f"Processing Record {record_count} of Set {set_count} i {city}")
    
    #Add 1 to the record count
    record_count+=1
    
#Run an API request for each of the cities

    try:
        #Parse the Json and retrieve data
        city_weather=requests.get(city_url).json()
        #Parse out the needed data
        city_lat=city_weather["coord"]["lat"]
        city_lng=city_weather["coord"]["lon"]
        city_max_temp=city_weather["main"]["temp_max"]
        city_humidity=city_weather["main"]["humidity"]
        city_clouds=city_weather["clouds"]["all"]
        city_wind=city_weather["wind"]["speed"]
        city_country=city_weather["sys"]["country"]
        city_description=city_weather["weather"][0]["description"]
        #convert the date to ISO standard
        #city_date=datetime.utcfromtimestamp(city_weather["dt"]).strftime("%Y-%m-%d %H:%M:%S")
        #Append the city information into city_Data list
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Current Description": city_description
                         })
    
    #If error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass
    
    
#Indicate that data loading is complete
print("----------------------------")
print("Data Retrieval Complete     ")
print("----------------------------")

Beginning Data Retrieval       
-------------------------------
Processing Record 1 of Set 1 i altamira
Processing Record 2 of Set 1 i pula
Processing Record 3 of Set 1 i pacific grove
Processing Record 4 of Set 1 i touros
Processing Record 5 of Set 1 i iqaluit
Processing Record 6 of Set 1 i nanortalik
Processing Record 7 of Set 1 i tiksi
Processing Record 8 of Set 1 i taolanaro
City not found. Skipping...
Processing Record 9 of Set 1 i mar del plata
Processing Record 10 of Set 1 i hilo
Processing Record 11 of Set 1 i quatre cocos
Processing Record 12 of Set 1 i ushuaia
Processing Record 13 of Set 1 i amapa
Processing Record 14 of Set 1 i rikitea
Processing Record 15 of Set 1 i east london
Processing Record 16 of Set 1 i cape town
Processing Record 17 of Set 1 i moultrie
Processing Record 18 of Set 1 i pontivy
Processing Record 19 of Set 1 i yerbogachen
Processing Record 20 of Set 1 i punta arenas
Processing Record 21 of Set 1 i bonthe
Processing Record 22 of Set 1 i albany
Processing 

In [19]:
#city_weather

In [21]:
#Convert the arrary of dictionaries to a Pandas DataFrame
city_data_df=pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,Altamira,-3.2033,-52.2064,83.52,57,98,0.98,BR,overcast clouds
1,Pula,44.8683,13.8481,76.57,78,0,4.61,HR,clear sky
2,Pacific Grove,36.6177,-121.9166,60.37,78,17,11.86,US,few clouds
3,Touros,-5.1989,-35.4608,76.98,82,62,16.42,BR,broken clouds
4,Iqaluit,63.7506,-68.5145,47.70,87,100,4.61,CA,overcast clouds
5,Nanortalik,60.1432,-45.2371,44.22,91,93,10.87,GL,overcast clouds
6,Tiksi,71.6872,128.8694,48.24,81,31,4.16,RU,scattered clouds
7,Mar Del Plata,-38.0023,-57.5575,54.09,58,100,4.61,AR,fog
8,Hilo,19.7297,-155.0900,87.28,89,100,3.44,US,overcast clouds
9,Quatre Cocos,-20.2078,57.7625,69.39,76,43,15.55,MU,scattered clouds


In [22]:
new_column_order=["City", "Country", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Current Description" ]
city_data_df=city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,Altamira,BR,-3.2033,-52.2064,83.52,57,98,0.98,overcast clouds
1,Pula,HR,44.8683,13.8481,76.57,78,0,4.61,clear sky
2,Pacific Grove,US,36.6177,-121.9166,60.37,78,17,11.86,few clouds
3,Touros,BR,-5.1989,-35.4608,76.98,82,62,16.42,broken clouds
4,Iqaluit,CA,63.7506,-68.5145,47.70,87,100,4.61,overcast clouds
5,Nanortalik,GL,60.1432,-45.2371,44.22,91,93,10.87,overcast clouds
6,Tiksi,RU,71.6872,128.8694,48.24,81,31,4.16,scattered clouds
7,Mar Del Plata,AR,-38.0023,-57.5575,54.09,58,100,4.61,fog
8,Hilo,US,19.7297,-155.0900,87.28,89,100,3.44,overcast clouds
9,Quatre Cocos,MU,-20.2078,57.7625,69.39,76,43,15.55,scattered clouds


In [23]:
len(city_data)

666

In [25]:
#Create the output file (csv)
output_data_file="Weather_Database/WeatherPy_Database.csv"
#Export the City_Data into a csv
city_data_df.to_csv(output_data_file, index_label="City_ID")